<h3 align="center">
实验五: 非平衡电桥的应用
</h3>

请将第一个 python 单元格中的内容复制，**和你的实验报告的完整照片**提交给一个大语言模型（我们建议使用 Google Gemini 2.5 Pro 或者等价逻辑能力的LLM），并用生成的 python 代码块替换该单元格中的实例数据点。

In [ ]:
"""
You are a lab agent, and your task is to extract data faithfully from the images provided. If no image is provided, please tell the user to upload photos of his or her lab report.

Read the following instructions carefully before you answer.
- The images should contain clear, handwritten text. If any part is unclear, ask the user to supply a clearer image or transcribe the unclear part.
- Pay special attention to the unit used by the user. Compare measure results with standard ones, and convert them if necessary. Automatically adjust magnitudes, and inform the user of the adjustments in text before the python snippet.
- When writing code you are expected to follow the given format strictly. If no warnings or failures occur, you should output a single python snippet, enclosed within triple backticks. You need not add extra comments to your answer.

Data related to NTC (aka temperature sensor):
- Resistance at 0°C, a value from 2000-10000 ohm. Extract to `ntc_r0: float`. Unit: ohm.
- Current for calibration, a value around 0.1mA. Extract to `ntc_ci: float`. Unit: mA.
- A table of temperature-voltage pairs. Extract to `ntc_temp: list[float]` and `ntc_volt: list[float]`. Unit: °C and mV.

Data related to Pt-100 (aka platinum resistance thermometer):
- Resistance at 0°C, a value around 100 ohm. Extract to `pt100_r0: float`. Unit: ohm.
- Current for calibration, a value around 1-10mA. Extract to `pt100_ci: float`. Unit: mA.
- A table of temperature-voltage pairs. Extract to `pt100_temp: list[float]` and `pt100_volt: list[float]`. Unit: °C and mV.
"""

# TODO: Replace the sample data below with the extracted data.

date = "2025-01-01"  # Example date, replace with actual extracted date

# NTC data
ntc_rp = 7000.0 # unit: ohm
ntc_ci = 0.13 # unit: mA
ntc_temp = [30.00, 40.00, 50.00, 60.00, 70.00, 80.00, 90.00, 100.00] # unit: °C
ntc_volt = [55.43, 64.04, 74.42, 83.80, 90.05, 95.05, 98.40, 100.63] # unit: mV

# Pt-100 data
pt100_rp = 100.00 # unit: ohm
pt100_ci = 5.000 # unit: mA
pt100_temp =  [30.00, 40.00, 50.00, 60.00, 70.00, 80.00, 90.00, 100.00] # unit: °C
pt100_volt = [17.20, 20.23, 25.46, 30.87, 35.75, 41.12, 45.68, 50.14] # unit: mV

In [ ]:
import me

info_box_data = {
    "姓名": me.get_name(),
    "学号": me.get_id(),
    "实验日期": date,
}

In [ ]:
from rich.table import Table
from rich.console import Console
from math import log

def get_resistance_from_bridge(ci_ma: float, volt_mv: float, r_std: float, r_p: float) -> float:
    return (
        ci_ma * 1e-3 * r_std * r_p + (2 * r_std + r_p) * volt_mv * 1e-3
    ) / (ci_ma * 1e-3 * r_std - volt_mv * 1e-3)

ntc_r12 = 1000

ntc_kelvin = [t + 273.15 for t in ntc_temp]
ntc_resistance_a = [
    get_resistance_from_bridge(ntc_ci, v, ntc_r12, ntc_rp) for v in ntc_volt
]
ntc_resistance_b = [
    get_resistance_from_bridge(ntc_ci, -v, ntc_r12, ntc_rp) for v in ntc_volt
]
ntc_resistance = ntc_resistance_a if (
    ntc_resistance_a[0] > ntc_resistance_a[-1]
) else ntc_resistance_b

ntc_t_inv = [1 / t for t in ntc_kelvin]
ntc_ln_r = [log(r) for r in ntc_resistance]

table = Table(title="NTC Data")
table.add_column("Temperature (K)", justify="right", style="cyan", no_wrap=True)
table.add_column("Voltage (mV)", justify="right", style="yellow")
table.add_column("Resistance (Ohm)", justify="right", style="white")
table.add_column("1/T (1e-3/K)", justify="right", style="white")
table.add_column("ln(R)", justify="right", style="white")
for t, v, r, t_inv, ln_r in zip(ntc_kelvin, ntc_volt, ntc_resistance, ntc_t_inv, ntc_ln_r):
    table.add_row(f"{t:.2f}", f"{v:.2f}", f"{r:.1f}", f"{1000 * t_inv:.3f}", f"{ln_r:.3f}")
console = Console()
console.print(table)

In [ ]:
import graphing # noqa: F401 - import for side effects
from graphing import plot_graph
import matplotlib.pyplot as plt
from rich import print

plot1_options = {
    "title": "NTC 电阻-温度特性曲线",
    "xlabel": "温度 ($K$)",
    "ylabel": r"电阻 ($\Omega$)",
    "xlim": (300, 380),
    "ylim": (0, 3200),
    "color": "black",
    "exclude_points": [],
    "linear_regression": False,
    "info_box": {
        "location": "upper left",
        "data": info_box_data
    }
}

plot2_options = {
    "title": "NTC 电阻对数-温度倒数特性曲线",
    "xlabel": "温度倒数 $1/T$ ($1/K$)",
    "ylabel": r"电阻对数 $\mathrm{ln}(R/\Omega)$",
    "xlim": (0.00265, 0.00330),
    "ylim": (4.8, 8.2),
    "color": "red",
    "exclude_points": [],
    "linear_regression": True,   # Enable regression for the second plot
    "regression_range": 0.9,
    "info_box": info_box_data
}

ntc_fig, (ntc_sub1, ntc_sub2) = plt.subplots(1, 2, figsize=(20, 8))
ntc_fig.suptitle('图1：NTC 电阻特性图', fontsize=32)

# Use the utility function to drive the plotting for each subplot
__ = plot_graph(ntc_sub1, ntc_kelvin, ntc_resistance, plot1_options)
lr = plot_graph(ntc_sub2, ntc_t_inv, ntc_ln_r, plot2_options)

# Show regression results
print(lr)
slope, intercept = lr["k"], lr["b"]
r_squared = lr["r_squared"]
print(f"[bold yellow]NTC 线性回归结果：[/bold yellow]材料系数 B = {slope:.2e} K, R² = {r_squared:.6f}")

# Adjust layout and show the plot
ntc_fig.tight_layout(pad=.5, rect=(0, 0, 1, 0.95)) # rect makes space for suptitle
plt.show()

In [ ]:
pt100_r12 = 100
pt100_kelvin = [t + 273.15 for t in pt100_temp]
pt100_resistance_a = [
    get_resistance_from_bridge(pt100_ci, v, pt100_r12, pt100_rp) for v in pt100_volt
]
pt100_resistance_b = [
    get_resistance_from_bridge(pt100_ci, -v, pt100_r12, pt100_rp) for v in pt100_volt
]
pt100_resistance = pt100_resistance_a if (
    pt100_resistance_a[0] < pt100_resistance_a[-1]
) else pt100_resistance_b

table = Table(title="Pt-100 Data")
table.add_column("Temperature (K)", justify="right", style="cyan", no_wrap=True)
table.add_column("Voltage (mV)", justify="right", style="yellow")
table.add_column("Resistance (Ohm)", justify="right", style="white")
for t, v, r in zip(pt100_kelvin, pt100_volt, pt100_resistance):
    table.add_row(f"{t:.2f}", f"{v:.2f}", f"{r:.2f}")
console = Console()
console.print(table)

In [ ]:
plot3_options = {
    "title": "",
    "xlabel": "温度 ($K$)",
    "ylabel": r"电阻 ($\Omega$)",
    "xlim": (30, 110),
    "ylim": (110, 142),
    "color": "black",
    "exclude_points": [0],
    "linear_regression": True,
    "info_box": {
        "location": "upper left",
        "offset": (0.0, -0.12),
        "data": info_box_data
    }
}

pt100_fig, pt100_sub = plt.subplots(1, 1, figsize=(10, 8))
pt100_fig.suptitle('图2：Pt-100 电阻特性图', fontsize=24)

lr = plot_graph(pt100_sub, pt100_temp, pt100_resistance, plot3_options)

# Show regression results
slope, intercept = lr["k"], lr["b"]
r_squared = lr["r_squared"]
print(f"[bold yellow]Pt-100 线性回归结果：[/bold yellow]斜率 B0 = {slope:.2e} Ω/K, 截距 A0 = {intercept:.2f} Ω, R² = {r_squared:.6f}")
x = ("对比 Re = R0 (1 + A0) 得到：") + '\n'
x += ("R0 = {:.1f} Ω".format(intercept)) + '\n'
x += ("A = B0 / R0 = {:.3e} K^{{-1}}".format(slope / intercept))
print(x)

pt100_fig.tight_layout(pad=0, rect=(0, 0, 1, 0.95)) # rect makes space for suptitle
plt.show()

In [ ]:
# Save the two figures to disk

ntc_fig.savefig("output/ntc_characteristic_curve.png", dpi=300)
pt100_fig.savefig("output/pt100_characteristic_curve.png", dpi=300)